**REKOMENDASI BUKU BERDASARKAN RATING**

In [4]:
# tambahkan Library Yang Dibutuhkan

import numpy as np
import tensorflow as tf
import pandas as pd
import pickle as pkl
import tensorflow.keras as keras
from tensorflow.keras.layers import Input
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Embedding, Flatten, Concatenate, Input
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [6]:
# membuat koneksi dengan dataset yang berada pada google drive

books_path = '/content/drive/My Drive/Dataset/Books.csv'
rate_path = '/content/drive/My Drive/Dataset/Ratings.csv'
user_path = '/content/drive/My Drive/Dataset/Users.csv'



books= pd.read_csv(books_path, error_bad_lines=False)
rate = pd.read_csv(rate_path, error_bad_lines=False)
user = pd.read_csv(user_path, error_bad_lines=False)

<ipython-input-6-84148d9b0f74>:9: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books= pd.read_csv(books_path, error_bad_lines=False)
<ipython-input-6-84148d9b0f74>:9: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books= pd.read_csv(books_path, error_bad_lines=False)
<ipython-input-6-84148d9b0f74>:10: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  rate = pd.read_csv(rate_path, error_bad_lines=False)
<ipython-input-6-84148d9b0f74>:11: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  user = pd.read_csv(user_path, error_bad_lines=False)


In [7]:
# Cek Tabel Dataset Buku dengan 5 kolom

books.head(5)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [8]:
# Cek Tabel Dataset Rating Dengan 5 Kolom

rate.head(5)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [9]:
# Cek Tabel Dataset User Dengan 5 Kolom

user.head(5)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [10]:
print("Buku     : " , books.shape)
print("Rating   : " , rate.shape)
print("Pengguna : " , user.shape)

Buku     :  (271360, 8)
Rating   :  (1149780, 3)
Pengguna :  (278858, 3)


In [11]:
# Cek Jumlah Atribut Pada Dataset Buku

books.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [12]:
# Cek Jumlah Atribut Pada Dataset Pengguna

user.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

Cek Apakah Ada Data Yang Terduplikat

In [13]:
rate.duplicated().sum()

0

In [14]:
user.duplicated().sum()

0

In [15]:
# Menggabungkan Dataset

ratings_with_name = rate.merge(books,on="ISBN")
ratings_with_name.shape

(1031136, 10)

In [16]:
ratings_with_name.head(5)

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...


In [17]:
# Preprocess data
n_users = rate['User-ID'].nunique()
n_books = rate['ISBN'].nunique()

user_ids = rate['User-ID'].astype("category").cat.codes.values
book_ids = rate['ISBN'].astype("category").cat.codes.values
ratings = rate['Book-Rating'].values

In [18]:
X_train_user, X_test_user, X_train_book, X_test_book, y_train, y_test = train_test_split(
    user_ids, book_ids, ratings, test_size=0.2, random_state=42
)

In [19]:
# Define the matrix factorization model using TensorFlow
embedding_size = 50  # Choose an appropriate embedding size
num_users = len(np.unique(user_ids))
num_books = len(np.unique(book_ids))

user_input = tf.keras.layers.Input(shape=(1,))
book_input = tf.keras.layers.Input(shape=(1,))

user_embedding = tf.keras.layers.Embedding(num_users, embedding_size)(user_input)
book_embedding = tf.keras.layers.Embedding(num_books, embedding_size)(book_input)



In [20]:
# Menambahkan Dense layer sebelum output
user_vecs = tf.keras.layers.Flatten()(user_embedding)
book_vecs = tf.keras.layers.Flatten()(book_embedding)

# Menambahkan Dense layer dengan aktivasi ReLU
dense_layer = tf.keras.layers.Dense(128, activation='relu')(tf.keras.layers.Concatenate()([user_vecs, book_vecs]))

# Output layer
output = tf.keras.layers.Dense(1, activation='linear')(dense_layer)

model = tf.keras.models.Model(inputs=[user_input, book_input], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error')


In [22]:
# Train the model with smaller model and batch size
model.fit(
    x=[X_train_user, X_train_book],
    y=y_train,
    epochs=10,
    batch_size=4096,
    validation_data=([X_test_user, X_test_book], y_test)
)

Epoch 1/10
225/225 [==============================] - 150s 666ms/step - loss: 13.2541 - val_loss: 11.5370
Epoch 2/10
225/225 [==============================] - 141s 626ms/step - loss: 9.6097 - val_loss: 11.9478
Epoch 3/10
225/225 [==============================] - 143s 634ms/step - loss: 8.0438 - val_loss: 12.5638
Epoch 4/10
225/225 [==============================] - 144s 641ms/step - loss: 7.2982 - val_loss: 12.8254
Epoch 5/10
225/225 [==============================] - 146s 650ms/step - loss: 6.8222 - val_loss: 13.0568
Epoch 6/10
225/225 [==============================] - 147s 654ms/step - loss: 6.4033 - val_loss: 13.1927
Epoch 7/10
225/225 [==============================] - 145s 642ms/step - loss: 6.0062 - val_loss: 13.4141
Epoch 8/10
225/225 [==============================] - 147s 652ms/step - loss: 5.6399 - val_loss: 13.6976
Epoch 9/10
225/225 [==============================] - 143s 637ms/step - loss: 5.3120 - val_loss: 13.9757
Epoch 10/10
225/225 [==============================] -

In [23]:
# Make predictions
predictions = model.predict([user_ids, book_ids])


35931/35931 [==============================] - 70s 2ms/step


In [24]:
# Example: Recommend books for user_id 1
user_id_to_recommend = 2
user_rated_books = rate[rate['User-ID'] == user_id_to_recommend]['ISBN'].unique()

In [25]:
# Filter out books that the user has already rated
unrated_books = rate[~rate['ISBN'].isin(user_rated_books)]['ISBN'].unique()

In [26]:
# Create input for the model
user_input_for_recommendation = np.full(len(unrated_books), user_id_to_recommend)

# Transform ISBNs to book_ids using the same encoding as in the training data
book_ids_for_recommendation, _ = pd.factorize(unrated_books)

# Make predictions for unrated books
recommendation_scores = model.predict([user_input_for_recommendation, book_ids_for_recommendation])

10643/10643 [==============================] - 19s 2ms/step


In [27]:
# Get top recommendations
top_recommendations_indices = recommendation_scores.flatten().argsort()[-5:][::-1]
top_recommendations_isbn = unrated_books[top_recommendations_indices]

In [28]:
# Display recommended books
recommended_books = books[books['ISBN'].isin(top_recommendations_isbn)][['Book-Title', 'Image-URL-L']]

# Print the list of recommended books
print(f"\n Rekomendasi Buku Untuk User {user_id_to_recommend} berdasarkan Rating :\n")

for i, (title, url) in enumerate(recommended_books[['Book-Title', 'Image-URL-L']].values, start=1):
    print(f"{i}. Book Title: {title}")
    print(f"   Image URL: {url}\n")



 Rekomendasi Buku Untuk User 2 berdasarkan Rating :

1. Book Title: Futureland
   Image URL: http://images.amazon.com/images/P/0446610739.01.LZZZZZZZ.jpg

2. Book Title: The Prodigal Daughter
   Image URL: http://images.amazon.com/images/P/0671422294.01.LZZZZZZZ.jpg

3. Book Title: Vittorio De Sica: Contemporary Perspectives (Toronto Italian Studies)
   Image URL: http://images.amazon.com/images/P/0802083811.01.LZZZZZZZ.jpg

4. Book Title: Tales from King Arthur (Wordsworth Collection)
   Image URL: http://images.amazon.com/images/P/1853261157.01.LZZZZZZZ.jpg

